In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import os
if(not os.path.exists(os.getcwd()+"/data")):
    os.mkdir(os.getcwd()+"/data")
LINKS_FILE = os.getcwd()+"/data/links.txt"
df_before_cleanup_path = os.getcwd()+"/data/df_before_cleanup.csv"
df_w_features_path = os.getcwd()+"/data/df_w_features.csv"
game_summaries_path = os.getcwd()+"/data/game_summaries.csv"
teams_path = os.getcwd()+"/data/teams.csv"
pitching_path = os.getcwd()+"/data/pitching.csv"
pitchers_path = os.getcwd()+"/data/pitchers.csv"
batting_path = os.getcwd()+"/data/batting.csv"
open(pitching_path, 'w').close()
open(pitchers_path, 'w').close()
open(batting_path, 'w').close()
open(game_summaries_path, 'w').close()
open(teams_path, 'w').close()
open(df_w_features_path, 'w').close()
open(LINKS_FILE, 'w').close()
open(df_before_cleanup_path, 'w').close()

In [ ]:
import warnings
%matplotlib inline
%load_ext autoreload
%autoreload 2

warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
import threading
import queue

In [ ]:
import scrape_in_code_folder as scrape

# Get links for individual games

In [ ]:
# fill the queue with dates that we need games from
day_q = queue.Queue(maxsize=0)

get_day = pd.to_datetime('2020-07-15')

# fill queue with all the dates until yesterday
yesterday = pd.datetime.now() - pd.Timedelta(days=1)
days = []
while get_day < yesterday:
    get_day = get_day + pd.Timedelta(days=1)
    if pd.to_datetime(get_day).month < 3:
        continue    # baseball doesn't happen before march
    if pd.to_datetime(get_day).month > 11:
        continue   # baseball doesn't happen in december
    day_q.put(get_day)
day_q.qsize()


In [ ]:
def do_work(q, lock):
    #start working through the queue
    while not q.empty():
        day = q.get()
        games = scrape.get_game_links(day)
        
        #some days have no games
        if len(games)==0:
            q.task_done()
            print(f"{day} no games.")
            continue
        
        #save the games to disk
        lock.acquire()
        with open(LINKS_FILE, "a") as myfile:
            for g in games: myfile.write(g+'\n')
        lock.release()
        
        q.task_done()
        print(f"{day} done.")

In [ ]:
num_threads = 25    # num of firefox windows
lock = threading.Lock()

#start the workers
for i in range(num_threads):
    worker = threading.Thread(target=do_work, args=(day_q,lock,))
    worker.setDaemon(True)
    worker.start()

#wait for workers to finish
day_q.join()

# parse individual game links

In [ ]:
q = queue.Queue()
with open(LINKS_FILE, 'r') as f:
    links = [x.strip() for x in f.readlines()] 
for l in links:
    q.put(l)
q.qsize()

In [ ]:
def do_work(q, lock):
    #start working through the queue
    while not q.empty():
        link = q.get()
        scrape.process_link(link,lock)
        q.task_done()

In [ ]:
num_threads = 25   # num of firefox windows
lock = threading.Lock()

#start the workers
for i in range(num_threads):
    worker = threading.Thread(target=do_work, args=(q,lock,))
    worker.setDaemon(True)
    worker.start()
        
#wait for workers to finish
q.join()